<a href="https://colab.research.google.com/github/adiojha629/TEWH_Malaria_Adi_Files/blob/master/FSRCNN_Image_Upscaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task Description: Upscaling Image Resolution with FSRCNN
Run the code chunk below to obtain your appropriate test and train datasets. You do not need to understand the code chunk below. The next section of text will explain what the variables and characteristics of your dataset are. 
<br>
<br>
Your task is to develop a neural network architecture called Fast Super-Resolution Convolutonal Neural Network (FSRCNN), which is a recently developed method used to upscale low-resolution images into high-resolution images. Your task will to upscale downsampled images that 32x32 pixels into high resolution images that are 128x128 pixels.

In [ ]:
# Import relevant packages
import numpy as np
import os
from shutil import copyfile
from zipfile import ZipFile

# Download NIH dataset zip file
!wget -nc ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip

# Extract images if not already extracted
ROOT_DIR = os.path.join("/", "content")
if not os.path.isdir("cell_images"):
    print("Extracting images...")
    with ZipFile(os.path.join("cell_images.zip"), "r") as zipObj:
        zipObj.extractall()
    print("Done!")

# Install and import relevant packages
import numpy as np
import os
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6 libxrender1
import cv2
from PIL import Image

# Create new folders to save rescaled images
if not os.path.isdir("RescaledSet"):
    os.mkdir("RescaledSet")
if not os.path.isdir("RescaledSet/Parasitized"):
    os.mkdir("RescaledSet/Parasitized")
if not os.path.isdir("RescaledSet/Uninfected"):
    os.mkdir("RescaledSet/Uninfected")

# Generate list of parasitized file names
ParasitizedFiles = os.listdir("cell_images/Parasitized/")
UninfectedFiles = os.listdir("cell_images/Uninfected/")

# Remove Thumb.db files
while 'Thumbs.db' in ParasitizedFiles: ParasitizedFiles.remove('Thumbs.db')   
while 'Thumbs.db' in UninfectedFiles: UninfectedFiles.remove('Thumbs.db')  

# Pre-allocate memory space for images
Parasitized = np.empty([2500,128,128,3])
Uninfected = np.empty([2500,128,128,3])

# Resize and load parasitized images
for i in range(2500):
    TempImage = cv2.imread('cell_images/Parasitized/'+ParasitizedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Parasitized[i,:,:,:] = ResizedImage

# Resize and load uninfected images
for i in range(2500):
    TempImage = cv2.imread('cell_images/Uninfected/'+UninfectedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Uninfected[i,:,:,:] = ResizedImage
    
print('Uninfected Dataset size is:',np.shape(Uninfected))
print('Parasitized Dataset size is:',np.shape(Parasitized))

# Generate dataset labels
ParasitizedLabels = np.repeat([[0,1]], 2500, axis=0)
UninfectedLabels = np.repeat([[1,0]], 2500, axis=0)
Labels = np.concatenate((ParasitizedLabels,UninfectedLabels), axis=0)

# Generate image dataset
Dataset = np.concatenate((Parasitized, Uninfected), axis=0)

# Generate 5-fold cross-validation groups
CVIndices = np.random.permutation(Dataset.shape[0])
TrainInd, TestInd = CVIndices[:4000], CVIndices[4000:]

# Generate train and test sets
from skimage.transform import rescale, resize, downscale_local_mean

TrainOut = Dataset[TrainInd,:]
TestOut = Dataset[TestInd,:]
TrainIn = np.zeros([np.shape(TrainOut)[0],32,32,3])
TestIn = np.zeros([np.shape(TestOut)[0],32,32,3])
for i in range(np.shape(TrainOut)[0]):
  TrainIn[i,:,:,:] = downscale_local_mean(TrainOut[i,:,:,:], (4,4,1))
for i in range(np.shape(TestOut)[0]):
  TestIn[i,:,:,:] = downscale_local_mean(TestOut[i,:,:,:], (4,4,1))

# Delete large irrelevant variables to minimize RAM usage
del Dataset
del Parasitized
del Uninfected

--2020-03-14 17:29:22--  ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip
           => ‘cell_images.zip’
Resolving lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)... 130.14.55.35, 2607:f220:41e:7055::35
Connecting to lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)|130.14.55.35|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /Open-Access-Datasets/Malaria ... done.
==> SIZE cell_images.zip ... 353452851
==> PASV ... done.    ==> RETR cell_images.zip ... done.
Length: 353452851 (337M) (unauthoritative)

cell_images.zip     100%[===================>] 337.08M  73.9MB/s    in 4.8s    

2020-03-14 17:29:32 (70.5 MB/s) - ‘cell_images.zip’ saved [353452851]



KeyboardInterrupt: ignored

# Overview of Variables to Work With
After running the previous code chunk, you should have four variables listed below. Note that the 1st dimension in these NumPy arrays are the number of images, while the 2nd and 3rd dimensions are the pixel resolutions. Lastly, the 4th dimension is the number of channels (RGB images have 3 channels). 

- ```TrainIn```: This variable contains 4000 images that have been downsampled from 128x128 pixels to 32x32 pixels. This is what will be used to train the neural network architecture.
- ```TrainOut```: This variable contains the corresponding original 4000 images that are 128x128 pixels (ie. not downsampled).
- ```TestIn```: This will be the testing set containing 1000 images that have been downsampled to 32x32 pixels.
- ```TestOut```: This will be the testing set containing the corresponding original 1000 images that have not been downsampled. 

In [ ]:
print('The dimensions of the training inputs are:',np.shape(TrainIn))
print('The dimensions of the training outputs are:',np.shape(TrainOut))
print('The dimensions of the testing inputs are:',np.shape(TestIn))
print('The dimensions of the testing outputs are:',np.shape(TestOut))

The dimensions of the training inputs are: (4000, 32, 32, 3)
The dimensions of the training outputs are: (4000, 128, 128, 3)
The dimensions of the testing inputs are: (1000, 32, 32, 3)
The dimensions of the testing outputs are: (1000, 128, 128, 3)


# Fast Super-Resolution Convolutional Neural Network (FSRCNN)

## Background
The Fast Super-Resolution Convolutoinal Neural Network (FSRCNN) was recently developed in just 2016 by a group of data scientists in Hong Kong (https://arxiv.org/pdf/1608.00367.pdf), and since then has received over 800 citations. Less than 0.01% of published papers have over 1000 citations. Point is? FSRCNN was a huge deal at the time. Key phrase: at the time. Despite it the young age of FSRCNN, it has already been widely replaced by other more advanced methods due to the rapidly growing field of deep learning (as per usual). 
<br>
<br>
With that being said, FSRCNN is the perfect neural network model to serve as a baseline for upscaling image resolution. It is also simple enough that creating and training this model will serve as a critical exercise before you guys move on to more advanced architectures. In the figure below, the SRCNN is the original predecessor of the FSRCNN, which as you see is just a simple three-layered convolutional neural network. FSRCNN is both faster, smaller, and more effective than SRCNN in virtually all scenarios, which is why we do not bother testing out SRCNN. The architecture you guys are interested in re-creating is the one shown in the second row in the figure below.

![](https://drive.google.com/uc?export=view&id=16e1QmIPJmvSLiKrcraLrxsJlX_a3lPkT)

We see that the FSRCNN architecture can be split into five general sections, described below:

- **Feature Extraction**: This part consists of a single convolutional layer with a 5x5 filter and a somewhat arbitrarily chosen number of filters. For now, create this layer with *d* = 56 filters. This layer essentially extracts valuable features from the low-resolution image that will be used later on to re-create the high-resolution image.
- **Shrinking**: In this step, we use a 1x1 convolutional filter to shrink that high feature dimension (*d* = 56) of the LR image into a low feature dimension (*s* = 16) of the HR image. This allows us to greatly reduce the number of parameters, hence resulting in a leaner and faster model.
- **Non-Linear Mapping**: In this section, we use a somewhat arbitrarily chosen number of identical 3x3 convolution layers with *s* = 12 filters. This section is the most influential part of the FSRCNN performance. A higher number of mapping layers allows for a higher level of feature complexity. Since our images are relatively simple, we shall only use four mapping layers. 
- **Expanding**: This section can be interpretted in some ways, as the opposite of the shrinking section. After creating non-linear mapping relationships, we expand the HR feature dimension back to a high number of features to allow for a clearer HR image. We use a 1x1 convolutional layer with back to the original *d* = 56 filters to go back to a high feature dimension.
- **Deconvolution**: This last section upsamples and aggregates the high feature dimensions generated by the previous expanding section. We use a 9x9 filter, with the number of filters equal to the number of image channels. Since our images are in RGB, we use 3 filters, one for each color channel. The stride length determines the upscaling factor of the image. Since in our case, we want to upscale an image from 32x32 to 128x128 - a factor of 4x - we want ```stride_length = (4,4)```. 
<br>
<br>
For ALL layers, we will implement zero padding so that the image dimensions remain the same up until the deconvolution layer, in which upscaling occurs. In addition, the activation function to be used will be parametric ReLU, which is similar to a ReLU function, except that the negative output is a trainable slope, rather than 0, as shown in the image below. 

![](https://drive.google.com/uc?export=view&id=1qUZFlIBn2_aPnT6pdFxIhfbId6CbSOLI)

In the code chunk below, we created a single convolution and deconvolution layer for demonstration, so that you guys may build the full model. In the ```Conv2D()``` function, there are several arguments:

- ```filters```: Specifies the number of filters to use
- ```kernel_size```: Specifies the size of the filters
- ```strides```: Specifies the stride of the filter. Default setting is (1,1)
- ```padding```: Set ```padding = 'same'``` for zero padding. 
- ```activation```: specifies activation function, if you do not specify one, you can specify it in the next layer, which is what was done in the example below with ```PRelu()```. 
- ```kernel_initializer```: How to randomize the weights. For this, just set ```kernel_initializer = 'he_normal'``` for all layers, as shown in the example below.
<br>
<br>
In the code chunk below, I already written the model compilation section for you. You can tweak it as desired (eg. changing optimizer, learning rate, batch size, epochs, etc). The most important thing here is creating the architecture itself though. 



In [ ]:
## Create FSRCNN architecture
from keras import optimizers
from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Dense, Activation
from keras.layers import Conv2D, MaxPooling2D, Input, ZeroPadding2D, Conv2DTranspose, merge 
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import image

#Feature Extraction
model = Sequential()
input_img = Input(shape=(32,32,3))
model = Conv2D(filters = 56, kernel_size = (5, 5), padding='same', kernel_initializer='he_normal')(input_img)
model = PReLU()(model)

#Shrink
model = Conv2D(filters = 16, kernel_size = (1, 1), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)

#Mapping
model = Conv2D(filters = 12, kernel_size = (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)
model = Conv2D(filters = 12, kernel_size = (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)
model = Conv2D(filters = 12, kernel_size = (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)
model = Conv2D(filters = 12, kernel_size = (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)

#Exapansion
model = Conv2D(filters = 56, kernel_size = (1, 1), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)

#Deconvolution
model = Conv2DTranspose(filters = 3, kernel_size = (9, 9), strides=(4, 4), padding='same')(model)
output_img = model

model = Model(input_img, output_img) #Create the model object
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False) #Training optimizer
model.compile(loss = "mean_squared_error", optimizer = adam, metrics=["mean_squared_error"]) #How we measure error

model.summary()


Using TensorFlow backend.







Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 56)        4256      
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 32, 32, 56)        57344     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 16)        912       
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 32, 32, 16)        16384     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 12)        1740      
_________________________________________________________________
p_re_lu_3 (PReLU)            (None, 32, 32, 12)       

## Training the FSRCNN Model
Now we use the ```model.fit()``` function to train our neural network. Note the following arguments:

- ```x```: This is the input features, which in this case is ```x = TrainIn```.
- ```y```: This is the output features, which in this case is ```y = TrainOut```.
- ```validation_data```: This is the testing set, which in this case is ```validation_data = (TestIn,TestOut)```.
- ```epochs```: Specifies the number of epochs to run. Try using 100 epochs.
- ```batch_size```: Specifies the batch size, which is the number of samples used to compute the gradient for weight updates. Try using 32.
- ```validation_freq```: Specifies how often to use the test set. Set ```validation_freq = 1``` so that it tests the model on the test set for every 1 epoch. 

## Evaluating the Performance of the FSRCNN Model
The ```model.predict()``` function simply takes in the test dataset, which in this case is ```TestIn```, and returns the output ```ModelOut```, which contains the 4D NumPy array containing the upscaled images. This is what you will use to calculate the MSE and PSNR later on.

In [ ]:
Results = model.fit(y=TrainOut, x=TrainIn, validation_data = (TestIn,TestOut), epochs=100, batch_size = 32, validation_freq=1)

ModelOut = model.predict(TestIn)




Train on 4000 samples, validate on 1000 samples
Epoch 1/100





4000/4000 [==============================] - 17s 4ms/step - loss: 1858.8184 - mean_squared_error: 1858.8184 - val_loss: 514.2066 - val_mean_squared_error: 514.2066
Epoch 2/100
4000/4000 [==============================] - 2s 569us/step - loss: 389.0431 - mean_squared_error: 389.0431 - val_loss: 309.1778 - val_mean_squared_error: 309.1778
Epoch 3/100
4000/4000 [==============================] - 2s 569us/step - loss: 276.7609 - mean_squared_error: 276.7609 - val_loss: 252.8007 - val_mean_squared_error: 252.8007
Epoch 4/100
4000/4000 [==============================] - 2s 571us/step - loss: 238.2954 - mean_squared_error: 238.2954 - val_loss: 226.4367 - val_mean_squared_error: 226.4367
Epoch 5/100
4000/4000 [==============================] - 2s 568us/step - loss: 217.0480 - mean_squared_error: 217.0480 - val_loss: 208.4104 - val_mean_squared_error: 208.4104
Epoch 6/100
4000/4000 [==============================] - 2s 554us/st

## Calculating MSE
The loss function that we will use to train the model will be **MSE**, or **mean squared error**, which is simply defined as:

$ \text{MSE } =  \frac{1}{n}\sum^n_{i=1}(\mathbf{\hat{Y}}-\mathbf{Y})^2$

Where $\mathbf{\hat{Y}}$ is the predicted image matrix and $\mathbf{Y}$ is the true image matrix, and we do elementwise matrix subtraction. $n$ is simple the number of data points, which in our case would be

$n = \text{# Images} \times \text{Pixel Res}^2 \times \text{# Channels}$

So for example, our test set contains 1000 RGB images that are 128x128 pixels with 3 channels per image (RGB), giving us $n = 49,152,000$. Now, for convenience, let us create a function called ```MSE(Predict = , True = , n = )```, where ```Predict``` is the model output, ```True``` is the original image, and ```n``` is the number of images. We want the output to just be the MSE.

## Calculate PSNR
Peak signal-to-noise ratio (PSNR) is the standard measurement used to gauge how effective an image upscaling method is. We cannot use this function as our loss function for our neural network however, since the loss function must be differentiable. With that being said, we still want to calculate this on our test dataset after the model is trained. The formula for PSNR is

$ \text{PSNR} = 10\cdot\text{log}_{10}\left(\frac{\text{MAX}^2_I}{\text{MSE}} \right) $

Where $\text{MAX}^2_I$ is the maximum possible pixel value for the image. Since our images are stored as uint8, the maximum value is $\text{MAX}^2_I = 256^2 = 65536$. Now, for convenience, let us create a function called ```PSNR(MSE = , MAX = )```, where ```MSE``` is the input MSE and ```MAX``` is the maximum pixel value, which in this case is 256. 

In [ ]:
import numpy as np

def MSE(Predict,Actual,n):
  differences = np.subtract(Predict, Actual)
  Result = (np.sum(np.power(differences,2))) / n
  return Result

def PSNR(MSE,MAX):
  Result = 10 * np.log10((MAX^2)/MSE)
  return Result

## Generating Baseline Images for Evaluation
Now, we want to create two new NumPy arrays. While we can find the MSE and PSNR of our model, there is no good way to measure its relative performance until we compare it to other baseline methods. We will create a NumPy array called ```RawLR``` that contains the 32x32 image upscaled to 128x128 without any interpolation or upscaling methods (ie. every pixel in the 32x32 is basically duplicated into a 4x4 to generate the 128x128). We will also create a NumPy array called ```Bicubic``` that contains the 32x32 image upscaled to 128x128 via bicubic interpolation. In the code below, I show you how this is performed on a single image, but we need to generate the 4D NumPy array containing all of the test images from ```TestIn```. 

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from skimage.transform import rescale, resize, downscale_local_mean

# No interpolation example
NoInterp_Control = []
for i in range(len(TestIn)):
  NormalExample = np.zeros([128,128,3])
  NormalExample = rescale(TestIn[i,:,:,:], (4,4,1), order = 0, anti_aliasing=False)
  NoInterp_Control.append(NormalExample)
#cv2_imshow(NoInterp_Control[2])

# Bicubic interpolation example (loop j = 3 for RGB channels)
BiCubic_Control = []
for i in range(len(TestIn)):
  BicubicExample = np.zeros([128,128,3])
  for j in range(3):
    BicubicExample[:,:,j] = cv2.resize(TestIn[i,:,:,j], dsize = (128,128), interpolation = cv2.INTER_CUBIC)
  BiCubic_Control.append(BicubicExample)

#cv2_imshow(BicubicExample)
#cv2_imshow(BiCubic_Control[11])

## Calculate MSE and PSNR of Raw Images, Bicubic Images, and FSRCNN Images
By the time you reach this section, you have already created functions for calculating MSE and PSNR. In addition, you have generated the three NumPy arrays for each of the three methods (no interpolation, bicubic, and FSRCNN images). Now the simple put is inputting these NumPy arrays to calculate the MSE and PSNR of each of these methods so that we can see how well the FSRCNN is performing. 

In [ ]:
# Calculate the MSE and PSNR of images with no interpolation 
print('Raw Image MSE:')
n = MSE(NoInterp_Control,TestOut,len(TestOut))
print(n)
print('Raw Image PSNR:')
n = PSNR(n,256)
print(n)

# Calculate the MSE and PSNR of images with bicubic interpolation
print('\nBicubic Image MSE:')
n = MSE(BiCubic_Control,TestOut,len(TestOut))
print(n)
print('Bicubic Image PSNR:')
n = PSNR(n,256)
print(n)

# Calculate the MSE and PSNR of images with FSRCNN
print('\nFSRCNN Image MSE:')
n = MSE(ModelOut,TestOut,len(TestOut))
print(n)
print('FSRCNN Image PSNR:')
n = PSNR(n,256)
print(n)

Raw Image MSE:
19912906.6565625
Raw Image PSNR:
-48.87514951993367

Bicubic Image MSE:
12370807.549792508
Bicubic Image PSNR:
-46.8077834475493

FSRCNN Image MSE:
4336414.517288307
FSRCNN Image PSNR:
-42.255110837032916
